# Advanced Model Architectures

It's time to get introduced to more advanced architectures! You will create an autoencoder to reconstruct noisy images, visualize convolutional neural network activations, use deep pre-trained models to classify images and learn more about recurrent neural networks and working with text as you build a network that predicts the next word in a sentence.

# (1) Tensors, layers and autoencoders

## Accesing Keras layers

```
# Accessing the first layer of a Keras model
first_layer = model.layers[0]
# Printing the layer, and its input, output and weights
print(first_layer.input)
print(first_layer.output)
print(first_layer.weights)
```

## What are tensors?

```
# Define a rank 2 tensor (2 dimensions)
T2 = [[1, 2, 3], 
        [4, 5, 6],
        [7, 8, 9]]
# Define a rank 3 tensor (3 dimensions)
T3 = [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9],
        
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18],
        
        [19, 20, 21],
        [22, 23, 24],
        [25, 26, 27]]
```

```
# Import Keras backend
import keras.backend as K
# Get the input and output tensors of a model layer
inp = model.layers[0].input
out = model.layers[0].output
# Function that maps layer inputs to outputs
inp_to_out = K.function([inp], [out])
# We pass and input and get the output we'd get in that first layer
print(inp_to_out([X_train]))
```

## it's time to introduce a new architecture

## Auto encoders!

<img src="image/Screenshot 2021-01-29 234520.png">

<img src="image/Screenshot 2021-01-29 234600.png">

## Autoencoder use cases

- Dimensionality reduction:
    - Smaller dimensional space representation of our inputs
- De-noising data:
    - If trained with clean data, irrelevant noise will be filtered out during reconstruction
- Anomaly detection:
    - A poor reconstruction will result when the model is fed with unseen inputs.

## Building a simple autoencoder

```
# Instantiate a sequential model
autoencoder = Sequential()
# Add a hidden layer of 4 neurons and an input layer of 100
autoencoder.add(Dense(4, input_shape=(100,), activation='relu'))
# Add an output layer of 100 neurons
autoencoder.add(Dense(100, activation='sigmoid'))
# Compile your model with the appropiate loss
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
```

## Breaking it into an encoder

```
# Building a separate model to encode inputs
encoder = Sequential()
encoder.add(autoencoder.layers[0])
# Predicting returns the four hidden layer neuron outputs
encoder.predict(X_test)
```

# Exercise I: It's a flow of tensors

If you have already built a model, you can use the `model.layers` and the `keras.backend` to build functions that, provided with a valid input tensor, return the corresponding output tensor.

This is a useful tool when we want to obtain the output of a network at an intermediate layer.

For instance, if you get the input and output from the first layer of a network, you can build an `inp_to_out` function that returns the result of carrying out forward propagation through only the first layer for a given input tensor.

So that's what you're going to do right now!

`X_test` from the **Banknote Authentication** dataset and its `model` are preloaded. Type `model.summary()` in the console to check it.

### Instructions

- Import `keras.backend` as `K`.
- Use the `model.layers` list to get a reference to the input and output of the first layer.
- Use `K.function()` to define a function that maps inp to out.
- Print the results of passing `X_test` through the 1st layer.


In [ ]:
# Import keras backend
import keras.backend as K

# Input tensor from the 1st layer of the model
inp = model.layers[0].input

# Output tensor from the 1st layer of the model
out = model.layers[0].output

# Define a function from inputs to outputs
inp_to_out = K.function([inp], [out])

# Print the results of passing X_test through the 1st layer
print(inp_to_out([X_test]))

# Exercise II: Neural separation

Put on your gloves because you're going to perform brain surgery!

Neurons learn by updating their weights to output values that help them better distinguish between the different output classes in your dataset. You will make use of the `inp_to_out()` function you just built to visualize the output of two neurons in the first layer of the **Banknote Authentication** `model` as it learns.

The `model` you built in chapter 2 is ready for you to use, just like `X_test` and `y_test`. Paste `show_code(plot)` in the console if you want to check `plot()`.

You're performing heavy duty, once all is done, click through the graphs to watch the separation live!

### Instructions

- Use the previously defined `inp_to_out()` function to get the outputs of the first layer when fed with `X_test`.
- Use the `model.evaluate()` method to obtain the validation accuracy for the test dataset at each epoch.


In [ ]:
for i in range(0, 21):
  	# Train model for 1 epoch
    h = model.fit(X_train, y_train, batch_size = 16, epochs = 1, verbose = 0)
    if i%4==0: 
      # Get the output of the first layer
      layer_output = inp_to_out([X_test])[0]
      
      # Evaluate model accuracy for this epoch
      test_accuracy = model.evaluate(X_test, y_test)[1] 
      
      # Plot 1st vs 2nd neuron output
      plot()

## test_accuracy plot
<img src="image/2021-01-30  001134.svg" width=40%> 

<img src="image/2021-01-30  001721.svg" width=40%>

<img src="image/2021-01-30  001734.svg" width=40%>

<img src="image/2021-01-30  001751.svg" width=40%>

<img src="image/2021-01-30  001805.svg" width=40%>

<img src="image/2021-01-30  001823.svg" width=40%>

# Exercise II: Building an autoencoder

Autoencoders have several interesting applications like anomaly detection or image denoising. They aim at producing an output identical to its inputs. The input will be compressed into a lower dimensional space, **encoded**. The model then learns to **decode** it back to its original form.

You will encode and decode the **MNIST** dataset of handwritten digits, the hidden layer will encode a 32-dimensional representation of the image, which originally consists of 784 pixels (28 x 28). The autoencoder will essentially learn to turn the 784 pixels original image into a compressed 32 pixels image and learn how to use that encoded representation to bring back the original 784 pixels image.

The `Sequential` model and `Dense` layers are ready for you to use.

Let's build an autoencoder!

### Instructions

- Create a `Sequential` model.
- Add a dense layer with as many neurons as the encoded image dimensions and `input_shape` the number of pixels in the original image.
- Add a final layer with as many neurons as pixels in the input image.
- Compile your `autoencoder` using `adadelta` as an optimizer and `binary_crossentropy` loss, then summarise it.


In [ ]:
# Start with a sequential model
autoencoder = Sequential()

# Add a dense layer with input the original image pixels and neurons the encoded representation
autoencoder.add(Dense(32, input_shape=(784, ), activation="relu"))

# Add an output layer with as many neurons as the orginal image pixels
autoencoder.add(Dense(784, activation = "sigmoid"))

# Compile your model with adadelta
autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')

# Summarize your model structure
autoencoder.summary()

# Exercise III: De-noising like an autoencoder

Okay, you have just built an `autoencoder` model. Let's see how it handles a more challenging task.

First, you will build a model that encodes images, and you will check how different digits are represented with `show_encodings()`. To build the encoder you will make use of your `autoencoder`, that has already being trained. You will just use the first half of the network, which contains the input and the bottleneck output. That way, you will obtain a 32 number output which represents the encoded version of the input image.

Then, you will apply your `autoencoder` to noisy images from `MNIST`, it should be able to clean the noisy artifacts.

`X_test_noise` is loaded in your workspace. The digits in this noisy dataset look like this:

<img src="image/model_chapter2_binary_classification.jfif">

Apply the power of the autoencoder!

### Instructions 1/2

- Build an `encoder` model with the first layer of your trained `autoencoder` model.
- Predict on `X_test_noise` with your `encoder` and show the results with `show_encodings()`.


In [ ]:
# Build your encoder by using the first layer of your autoencoder
encoder = Sequential()
encoder.add(autoencoder.layers[0])

# Encode the noisy images and show the encodings for your favorite number [0-9]
encodings = encoder.predict(X_test_noise)
show_encodings(encodings, number = 1)

### Instructions 2/2

- Predict on `X_test_noise` with your `autoencoder`, this will effectively perform both the encoding and decoding.
- Plot noisy vs decoded images with `compare_plot()`.


In [ ]:
# Build your encoder by using the first layer of your autoencoder
encoder = Sequential()
encoder.add(autoencoder.layers[0])

# Encode the noisy images and show the encodings for your favorite number [0-9]
encodings = encoder.predict(X_test_noise)
show_encodings(encodings, number = 1)

# Predict on the noisy images with your autoencoder
decoded_imgs = autoencoder.predict(X_test_noise)

# Plot noisy vs decoded images
compare_plot(X_test_noise, decoded_imgs)

## Plot noisy vs Decoded

<img src="image/2021-01-30  005014.svg">